In [7]:
using Gridap
using LinearAlgebra
using SparseArrays
using GridapGmsh

#Geometria
R = 0.25  # Radio del rodillo (m)
Rl = 0.001 # Espesor de la lamina (m)
w = 1.345 # Ancho de la lámina (m)
w2 = w/2 # ancho medio (m)
H = 0.125 # Espesor de lamina (m)

#DOMINIOS






Definicion del dominio: Ancho = 0.6725 m, alto = 0.125 m


0.2

In [ ]:
using Gmsh

# ----------------------------
# Parámetros geométricos (Schneider)
R  = 0.25          # Radio del rodillo (m)
W  = 1.345         # Ancho total lámina (m)
W2 = W/2.0         # Mitad del ancho (0.6725 m)
H  = 0.125         # Altura total (m)

# Centro del rodillo en coordenadas globales
x_c_global = W2 + R        # 0.9225 m
y_c_global = H/2.0         # 0.0625 m

# --- Origen en la parte superior del rodillo (todo positivo) ---
origin_x = x_c_global - R  # punto más izquierdo del rodillo => 0
origin_y = 0.0             # top (entrada) => 0

# Centro del rodillo en coordenadas locales (trasladado al nuevo origen)
x_c = x_c_global - origin_x
y_c = y_c_global - origin_y

# Mallado
lc = 0.005         # Tamaño característico de los elementos
npts_arc = 60      # Resolución del arco
# ----------------------------

# Inicializar Gmsh
Gmsh.initialize()
Gmsh.option.setNumber("General.Terminal", 1)
Gmsh.model.add("dominio_schneider")

# ----------------------------
# 1) Semicircunferencia (lado derecho del rodillo)
angles = range(-π/2, π/2, length=npts_arc)
pt_ids = Int[]
for θ in angles
    x = x_c + R*cos(θ)   # lado derecho
    y = y_c + R*sin(θ)
    push!(pt_ids, Gmsh.model.occ.addPoint(x, y, 0.0, lc))
end

# ----------------------------
# 2) Puntos clave
p_sym_top   = Gmsh.model.occ.addPoint(0.0, 0.0, 0.0, lc)   # Simetría superior
p_sym_bot   = Gmsh.model.occ.addPoint(0.0, H, 0.0, lc)     # Simetría inferior
p_trans_top = Gmsh.model.occ.addPoint(W2 - origin_x, 0.0, 0.0, lc) # transición superior

# ----------------------------
# 3) Crear líneas
line_ids = Int[]
for i in 1:(length(pt_ids)-1)
    push!(line_ids, Gmsh.model.occ.addLine(pt_ids[i], pt_ids[i+1]))
end

l_trans = Gmsh.model.occ.addLine(pt_ids[end], p_trans_top)  # transición rodillo -> top
l_top   = Gmsh.model.occ.addLine(p_trans_top, p_sym_top)    # borde superior
l_sym   = Gmsh.model.occ.addLine(p_sym_top, p_sym_bot)      # eje de simetría
l_bot   = Gmsh.model.occ.addLine(p_sym_bot, pt_ids[1])      # borde inferior

all_lines = vcat(line_ids, [l_trans, l_top, l_sym, l_bot])

# ----------------------------
# 4) Superficie cerrada
cloop = Gmsh.model.occ.addCurveLoop(all_lines)
surf  = Gmsh.model.occ.addPlaneSurface([cloop])
Gmsh.model.occ.synchronize()

# ----------------------------
# 5) Etiquetas físicas
Gmsh.model.addPhysicalGroup(2, [surf], 100)       # dominio
Gmsh.model.addPhysicalGroup(1, [l_sym], 1)        # 1 = simetría
Gmsh.model.addPhysicalGroup(1, [l_top], 2)        # 2 = entrada superior
Gmsh.model.addPhysicalGroup(1, [l_bot], 3)        # 3 = salida inferior
Gmsh.model.addPhysicalGroup(1, line_ids, 4)       # 4 = rodillo (arco)
Gmsh.model.addPhysicalGroup(1, [l_trans], 5)      # 5 = transición piscina-rodillo

# ----------------------------
# 6) Mallar y guardar
Gmsh.model.mesh.generate(2)
Gmsh.write("dominio_marialaura.msh")
println("✅ Malla guardada: dominio_marialaura.msh")

Gmsh.finalize()